### docs

In [ ]:
https://python.langchain.com/docs/integrations/chat/ollama/

https://python.langchain.com/docs/integrations/llms/ollama/
https://python.langchain.com/docs/concepts/chat_models/
https://python.langchain.com/docs/concepts/structured_outputs/
https://python.langchain.com/docs/concepts/tokens/
https://python.langchain.com/docs/integrations/document_loaders/
https://python.langchain.com/docs/integrations/document_loaders/microsoft_word/
https://python.langchain.com/docs/integrations/document_loaders/image_captions/
https://python.langchain.com/docs/integrations/document_loaders/image/
https://python.langchain.com/docs/integrations/document_loaders/microsoft_excel/
https://python.langchain.com/docs/integrations/document_loaders/recursive_url/
https://python.langchain.com/docs/integrations/memory/



### install

In [ ]:
%pip install -qU langchain-ollama
%pip install -U ollama

### imports

In [4]:
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate



### basic lllm call

In [ ]:

llm = ChatOllama(
    model="llama3.1",
    temperature=0,
    # other params...
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)



### Tool calling

In [ ]:
from typing import List

from langchain_core.tools import tool
from langchain_ollama import ChatOllama


@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True


llm = ChatOllama(
    model="llama3.1",
    temperature=0,
).bind_tools([validate_user])

result = llm.invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)
result.tool_calls

In [ ]:
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
import requests

# Define a function to retrieve the exchange rate from API
@tool
def get_exchange_rate() -> str:
    """Retrieve the USD-BRL exchange rate"""
    try:
        response = requests.get('https://api.exchangerate-api.com/v4/latest/USD')
        data = response.json()
        print(data)
        brl_rate = data['rates']['BRL']
        print(brl_rate)
        return f"1 USD is equal to {brl_rate} BRL"
    except Exception as e:
        return str(e)

# Define the main function
@tool
def get_current_dollar_rate() -> str:
    """Get the current dollar rate using API"""
    return get_exchange_rate()

# Initialize the ChatOllama model and bind it with tools
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
).bind_tools([get_current_dollar_rate])

# Invoke the tool to get the current dollar rate
result = llm.invoke("What is the USD-BRL exchange rate today?")
print(result)
print(result.tool_calls)

#### current USD to BRL

http://localhost:3000/c/dbc115c4-ecfe-4322-adc6-dee923b7f296

https://api.bcb.gov.br/dados/serie/bcdata.sgs.10813/dados/ultimos/1?formato=json

[{"data":"10/01/2025","valor":"6.0959"}]

In [ ]:
from typing import List
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
import requests

# Função para buscar a taxa de câmbio
@tool
def get_exchange_rate() -> str:
    """Get the current USD to BRL exchange rate from the Brazilian Central Bank API."""
    api_url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.10813/dados/ultimos/1?formato=json"

    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Verifica se a resposta é 200 OK
        
        data = response.json()
        
        if data:
            exchange_rate = data[0]["valor"]
            return f"USD = {exchange_rate} BRL"
        else:
            return "No data available."
    
    except requests.exceptions.RequestException as e:
        return f"Error fetching exchange rate: {e}"

# Configuração do LLM
llm = ChatOllama(
    model="llama3.1",
    temperature=0,
).bind_tools([get_exchange_rate])

# Invoca a pergunta para o LLM, que usa a função 'get_exchange_rate'
result = llm.invoke(
    "What is the current USD to BRL exchange rate?"
)

# Acessando o resultado dos tool_calls de forma robusta
tool_results = [
    call.get('output', 'No output available') for call in result.tool_calls
]
print(tool_results)



In [ ]:
from typing import List

from langchain_core.tools import tool
from langchain_ollama import ChatOllama

# Função para buscar a taxa de câmbio
@tool
def get_exchange_rate() -> str:
    """Get the current USD to BRL exchange rate from the Brazilian Central Bank API."""
    api_url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.10813/dados/ultimos/1?formato=json"

    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Verifica se a resposta é 200 OK
        
        data = response.json()
        
        if data:
            exchange_rate = data[0]["valor"]
            print(exchange_rate)
            return f"USD = {exchange_rate} BRL"
        else:
            return "No data available."
    
    except requests.exceptions.RequestException as e:
        return f"Error fetching exchange rate: {e}"

llm = ChatOllama(
    model="llama3.2",
    temperature=0.8,
).bind_tools([get_exchange_rate])

result = llm.invoke(
    "Could let meknow the current exchange rate between USD and BRL?"
)
result.tool_calls